In [28]:
from random import randint
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras.callbacks import ModelCheckpoint
import numpy as np
from keras.models import model_from_yaml
from random import randint

In [42]:
# used help from https://machinelearningmastery.com/develop-character-based-neural-language-model-keras/ and 
# and https://github.com/vivshaw/shakespeare-LSTM

with open("data/shakespeare_LSTM.txt") as file:
    poems = file.read()

# Encoding and decoding
chars = sorted(list(set(poems)))
num_chars = len(chars)
encoding = {c: i for i, c in enumerate(chars)}
decoding = {i: c for i, c in enumerate(chars)}
print("There are {0} unique characters in the list of poems".format(num_chars))


There are 61 unique characters in the list of poems


In [66]:
sentence_length = 50
skip = 1
X_data = []
y_data = []
for i in range (0, len(poems) - sentence_length, skip): 
    sentence = poems[i:i + sentence_length]
    next_char = poems[i + sentence_length]
    X_data.append([encoding[char] for char in sentence])
    y_data.append(encoding[next_char])

num_sentences = len(X_data)
print('Dimension of X input data is {0}'.format(num_sentences))

X = np.zeros((num_sentences, sentence_length, num_chars), dtype=np.bool)
y = np.zeros((num_sentences, num_chars), dtype=np.bool)
for i, sentence in enumerate(X_data):
    for t, encoded_char in enumerate(sentence):
        X[i, t, encoded_char] = 1
    y[i, y_data[i]] = 1

Dimension of X input data is 94394


In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(sentence_length, num_chars)))
model.add(Dense(num_chars))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

architecture = model.to_yaml()
with open('model.yaml', 'a') as model_file:
    model_file.write(architecture)

# Set up checkpoints
file_path="weights-{epoch:02d}-{loss:.3f}.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor="loss", verbose=1, save_best_only=True, mode="min")
callbacks = [checkpoint]

model.fit(X, y, batch_size=128, epochs=30, callbacks=callbacks)

Loaded a corpus of 94444 characters
Our corpus contains 61 unique characters.
Sliced our corpus into 94394 sentences of length 50
Vectorizing X and y...
Sanity check y. Dimension: (94394, 61) # Sentences: 94394 Characters in corpus: 61
Sanity check X. Dimension: (94394, 50, 61) Sentence length: 50
Let's build a brain!
Epoch 1/30
738/738 [==============================] - 51s 68ms/step - loss: 2.9202

Epoch 00001: loss improved from inf to 2.60381, saving model to weights-01-2.604.hdf5
Epoch 2/30
738/738 [==============================] - 50s 68ms/step - loss: 2.1771

Epoch 00002: loss improved from 2.60381 to 2.12124, saving model to weights-02-2.121.hdf5
Epoch 3/30
738/738 [==============================] - 50s 67ms/step - loss: 1.9961

Epoch 00003: loss improved from 2.12124 to 1.97303, saving model to weights-03-1.973.hdf5
Epoch 4/30
738/738 [==============================] - 50s 67ms/step - loss: 1.8878

Epoch 00004: loss improved from 1.97303 to 1.87773, saving model to weights-04

In [22]:
def tag_seed(seed):
    # Grab a chunk of three words
    word_list = seed.split()
    i = randint(1, len(word_list) - 3)

    bad_start_end = set(['on', 'of', 'from', "I", "O!", "and", "be", 'or', 'the', 'than', 'with', 'by'])
    bad_start = set(['of'])
    bad_end = set(['no', 'an', 'if'])

    words = []
    for i, word in enumerate(word_list[i:i + 3]):
        if not word == "I" and not word == "O!":
            word = word.strip("',.;-!:?").lower()
        if i == 0 and word not in bad_start_end | bad_start:
            words.append(word)
        if i == 1:
            words.append(word)
        if i == 2 and word not in bad_start_end | bad_end:
            words.append(word)

    tag = " ".join(words)
    return tag

def format_sonnet(text):
    formatted = text.split("\n")

    # The first and last line cut off in the middle, so we'll ditch them
    formatted = formatted[1:len(formatted) - 1]

    # Eliminate empty strings, strings that are just newlines, or other improper strings
    formatted = [string for string in formatted if len(string) > 3]

    # Put a period on our last string, replacing other punctuation if it's there.
    if formatted[-1][-1].isalnum():
        formatted[-1] += "."
    else:
        formatted[-1] = formatted[-1][:-1] + "."

    return formatted

In [75]:
def generate(seed_pattern):
    X = np.zeros((1, sentence_length, num_chars), dtype=np.bool)
    print(X.shape)
    for i, character in enumerate(seed_pattern):
        X[0, i, encoding[character]] = 1


    generated_text = ""
    for i in range(650):
        prediction = np.argmax(model.predict(X, verbose=0))
        generated_text += decoding[prediction]

        activations = np.zeros((1, 1, num_chars), dtype=np.bool)
        activations[0, 0, prediction] = 1
        X = np.concatenate((X[:, 1:, :], activations), axis=1)

    return generated_text

def make_seed(seed_phrase="When of the world of live thee all thee,"):
    if seed_phrase:
        phrase_length = len(seed_phrase)
        pattern = ""
        for i in range (0, sentence_length):
            pattern += seed_phrase[i % phrase_length]
    else:
        poems_length = len(poems)
        seed = randint(0, poems_length - sentence_length)
        pattern = poems[seed:seed + sentence_length]

    return pattern

In [76]:

seed_tag = make_seed()
print(seed_tag)
print('------------- Generated Poem ----------')
print(generate(seed_tag))

When of the world of live thee all thee,When of th
------------- Generated Poem ----------
(1, 50, 61)
ee.

If the dear received then do mine eyes,
The earth of thine and thine and thine of thee,
Who lov'st thou thou shouldst thou shalt strangets,
  The pays the brave of strange all the will,
  That in the brave the black as infored lies.

Those survey in thee, and then by delight
To me a says of thee that I do belovest,
When thou art the store to make the winter's part,
  And then be that summer's love in lovely grow.

The ear the world my mind in heaven still,
The endered meretion of the restor,
When sought the store to come not fair
That my great reported than my love strange:
Then who fair a seeth the still decease,
When your beauty shall 
